## Day 15

In [60]:
import re

def parse15(filename):
    with open(filename) as f:
        data = [ [ int(m) for m in re.findall(r"[-]?\d+",l) ] for l in f.readlines() ] 
        return data

### Part 1

#### Using sets...

... even if I already suspect this won't scale for part 2!

In [193]:
def coverageAtYset(data,Y=10):
    coverage = set()
    for m in data:
        S = (m[0],m[1])
        B = (m[2],m[3])
        d = Manhattan(S,B)
        dy = abs(Y-S[1])
        if dy<=d: # Y intercept the sensor area
            xmin = S[0]-(d-dy)
            xmax = S[0]+(d-dy)
            for x in range(xmin,xmax+1):
                coverage.add(x)
    return coverage

def part1set(data,Y=10):
    cov = coverageAtYset(data,Y)
    # remove any exisiting beacon from coverage at Y
    for m in data:
        B = (m[2],m[3])
        if B[1]==Y and B[0] in cov:
            cov.remove(B[0])
    return len(cov)

In [194]:
data0 = parse15("examples/example15.txt")
data = parse15("AOC2022inputs/input15.txt")

In [195]:
print("Test 1:",part1set(data0,Y=10))     # 26
print("Part 1:",part1set(data,Y=2000000)) # 5564017

Test 1: 26
Part 1: 5564017


#### Using ranges...

... to speed it up for Part 2

In [216]:
def overlap(r1,r2):
    s1,e1 = r1
    s2,e2 = r2
    if s1<=e2 and (e1>=s2 or s2==e1+1) or s2<=e1 and (e2>=s1 or s1==e2+1) : # overlap or superposition or contingous
        rnew = [min(s1,s2),max(e1,e2)]
        return [rnew]
    else: # disjoint
        return sorted([r1,r2],key=lambda x:x[0])

def coverageAtYrange(data,Y=10):
    coverage = []
    for m in data:
        S = (m[0],m[1])
        B = (m[2],m[3])
        d = Manhattan(S,B)
        dy = abs(Y-S[1])
        if dy<=d: # Y intercept the sensor area
            xmin = S[0]-(d-dy)
            xmax = S[0]+(d-dy)
            cov = [xmin,xmax]
            if not len(coverage):
                coverage.append(cov)
            else:
                newcoverage = []
                for c in coverage:
                    o = overlap(c,cov)
                    if len(o)==1:
                        cov = o[0]
                    else:
                        newcoverage.append(o[0])
                        cov = o[1]
                newcoverage.append(cov)
                coverage = sorted(newcoverage,key=lambda x:x[0])    
    return coverage

print(sorted(coverageAtYset(data0,Y=11)))
print(coverageAtYrange(data0,Y=11))

[-3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
[[-3, 13], [15, 25]]


In [232]:
def part1range(data,Y=10):
    cov = coverageAtYrange(data,Y)[0]
    noBeacons = cov[1]-cov[0]+1
    # remove any exisiting beacon(s) from coverage at Y
    removedBeacons = set()
    for m in data:
        B = (m[2],m[3])
        if B[1]==Y and cov[0]<=B[0]<=cov[1]+1 and B not in removedBeacons:
            noBeacons -= 1
            removedBeacons.add(B)
    return noBeacons

In [233]:
print("Test 1:",part1range(data0,Y=10))     # 26
print("Part 1:",part1range(data,Y=2000000)) # 5564017

Test 1: 26
Part 1: 5564017


### Part 2

In [250]:
def part2(data,ymin=0,ymax=20):
    print("Searching for hole in sensor coverages... ")
    for y in range(ymin,ymax+1):
        if y%100000==0:
            print(y,end=" ")
        cov = coverageAtYrange(data,Y=y)
        if len(cov)>1:
            print("\nFound!")
            x = cov[0][1]+1
            print("x = {}, y = {} --> ".format(x,y), end="")
            tf = x*4000000+y
            print("tuning frequency = {}".format(tf))
            return tf

In [251]:
part2(data0,ymin=0,ymax=20)

Searching for hole in sensor coverages... 
0 
Found!
x = 14, y = 11 --> tuning frequency = 56000011


56000011

In [252]:
part2(data,ymin=0,ymax=4_000_000)

Searching for hole in sensor coverages... 
0 100000 200000 300000 400000 500000 600000 700000 800000 900000 1000000 1100000 1200000 1300000 1400000 1500000 1600000 1700000 1800000 1900000 2000000 2100000 2200000 2300000 2400000 2500000 2600000 2700000 2800000 2900000 3000000 3100000 3200000 3300000 
Found!
x = 2889605, y = 3398893 --> tuning frequency = 11558423398893


11558423398893